Carlos,
      In the dataset, there are conversations between workers and ChatGPT or between workers and lawyers on Zoom. I would like to ask you for help with the following:

1) **Separate what text corresponds to the worker and which to the IA (or the lawyer).**

2) Extract measures of quality from the conversation.

**a) How complete was the response, did it cover many aspects of the question?**
b) How easy to understand was the response
c) Relevant:  That is: did it actually respond to the question asked?
d) How empathic the response was, that is:
was the name of the worker mentioned? how many times?
Was it friendly
Did the IA or lawyer made the worker feel understood?
e) How long did the conversation last (not possible now since we only have duration for the lawyer zoom data)


Thanks!!
Enirque

Catálogo de preguntas sugeridas
ID y pregunta. El ID es el que aparece en las columnas preguntas mostradas y preguntas seleccionadas de los reportes.

  1 | ¿Cuéntame cómo va a ser la audiencia de conciliación?
  2 | ¿Cuál debería de ser mi estrategia en la audiencia?
  3 | ¿Cuál es la diferencia entre un despido justificado y uno injustificado?
  4 | ¿Cuánto tiempo tengo para exigir mis derechos?
  5 | ¿Cuánto tiempo tengo para demandar después de obtener mi constancia de no conciliación?
  6 | ¿Qué prestaciones o pagos me corresponden por despido injustificado?
  7 | ¿A qué centro de conciliación debo acudir si tengo un conflicto laboral?
  8 | ¿Qué debo decir y qué no debo decir en la audiencia de conciliación?
  9 | ¿Me recomiendas llevar un abogado a la audiencia de conciliación?
 10 | ¿Qué sucede si no se llega a un acuerdo en la audiencia de conciliación?
 11 | ¿Qué pasa si mi patrón no me paga en la fecha acordada en el convenio de conciliación?

In [ ]:
#Se usó gpt 4.0
#Usar embbedings

#Respuestas empáticos

#Agregar métricas de las palabras

#En qué gana uno y en qué gana otro

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import json
import requests
import os
import pickle
from google.colab import auth, drive
import time
auth.authenticate_user()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Ai
df = pd.read_stata("gdrive/MyDrive/Conciliación laboral/conversations_ai.dta")

In [ ]:
#Abogados
df2 = pd.read_stata("gdrive/MyDrive/Conciliación laboral/conversations_human.dta")

*Calidad respuestas Ai*

In [ ]:
import time
prueba_path="gdrive/MyDrive/Conciliación laboral/"
carpetas=os.listdir(prueba_path)
todos=os.listdir(prueba_path+"/Conversaciones ai/")
print(len(todos))

619


In [ ]:
#Lista de grabaciones listas y faltantes
grabaciones = []
for grab in todos:
  grabaciones.append(grab.split(".")[0])
grabaciones_listas = [f[:-5] for f in os.listdir("gdrive/MyDrive/Conciliación laboral/Conversaciones ai") if f.endswith(".json")]
grabaciones_faltantes = list(set(grabaciones) - set(grabaciones_listas))

In [ ]:
print(len(grabaciones) , len(grabaciones_listas) , len(grabaciones_faltantes))

619 619 0


In [ ]:
prueba = pd.read_json(f"gdrive/MyDrive/Conciliación laboral/Conversaciones ai/{grabaciones[0]}.json")

In [ ]:
grabaciones[0]

'480'

In [ ]:
df[df["id"] == 480]

,id,fecha_creacion,fecha_inicio_sesion,fecha_fin_sesion,motivo_finalizacion,duracion_sesion,numero_preguntas,preguntas_mostradas,preguntas_seleccionadas,conversacion,session_id
0,480,10/21/2025 12:26,10/21/2025 12:26,10/21/2025 12:26,usuario,00:00:00,1,"[4,10,7,2]",[4],### Mensaje de la Persona\n\n¿Cuánto tiempo te...,564.0


In [ ]:
prueba

,iteracion,PParticipante,id_sesion,texto
0,1,persona,596,¿Cuéntame cómo va a ser la audiencia de concil...
1,2,ai,596,💼 **Audiencia de Conciliación: Un Entorno de D...
2,3,unknown,596,1. **Presentación:** \n - El conciliador rev...
3,4,persona,596,Se considera despido si constantemente me amen...
4,5,ai,596,"Es comprensible que te sientas preocupado, Mar..."
5,6,persona,596,¿Qué sucede si no se llega a un acuerdo en la ...
6,7,ai,596,"María, es importante saber qué ocurre si la co..."
7,8,persona,596,¿Cuál es la diferencia entre un despido justif...
8,9,ai,596,Entender la diferencia entre un despido justif...
9,10,persona,596,Tengo contrato anual por cinco años que procede


In [ ]:
df[df["session_id"] == 596]

,id,fecha_creacion,fecha_inicio_sesion,fecha_fin_sesion,motivo_finalizacion,duracion_sesion,numero_preguntas,preguntas_mostradas,preguntas_seleccionadas,conversacion,session_id
480,939,11/6/2025 11:16,11/6/2025 11:16,11/6/2025 11:30,timeout,00:14:00,6,"[10,3,7,1]","[1,10,3,7]",### Mensaje de la Persona\n\n¿Cuéntame cómo va...,596.0


# Funciones

In [ ]:
import pandas as pd
import unicodedata
import re
from typing import List, Union, Dict

def _normalize(text: str) -> str:
    """Lowercase + quitar acentos."""
    if pd.isna(text):
        return ""
    s = str(text).lower()
    s = ''.join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    return s

def detectar_nombres_dicc(
    df: pd.DataFrame,
    nombres: Union[List[str], Dict[str, List[str]]],
    texto_col: str = "texto",
    iter_col: str = "iteracion",
) -> Dict[str, List]:
    """
    Devuelve un diccionario SOLO con los nombres que SÍ aparecieron:
    {
        "nombres": [...],
        "conteos": [...],
        "iteraciones": [...]
    }
    """

    # Convertir lista → dict canonical->alias list
    if isinstance(nombres, list):
        names_dict = {n: [n] for n in nombres}
    else:
        names_dict = {k: (v if isinstance(v, list) else [v]) for k, v in nombres.items()}

    # Normalizar nombres y compilar regex
    regex_map = {}
    for canonical, aliases in names_dict.items():
        aliases_norm = [_normalize(a) for a in aliases]
        alternation = "|".join(
            re.escape(a) for a in sorted(set(aliases_norm), key=len, reverse=True)
        )
        pattern = re.compile(rf"\b(?:{alternation})\b", flags=re.IGNORECASE)
        regex_map[canonical] = {"pattern": pattern}

    # Estructura acumuladora
    resumen = {canonical: {"count": 0, "iters": []} for canonical in regex_map}

    # Recorrer la conversación fila por fila
    for idx, row in df.iterrows():
        texto_norm = _normalize(row.get(texto_col, ""))
        try:
            iter_actual = int(row.get(iter_col, idx + 1))
        except:
            iter_actual = int(idx + 1)

        for canonical, info in regex_map.items():
            matches = info["pattern"].findall(texto_norm)
            if matches:
                n = len(matches)
                resumen[canonical]["count"] += n
                resumen[canonical]["iters"].extend([iter_actual] * n)

    # Construir el diccionario final SOLO con nombres detectados
    out = {
        "nombres": [],
        "conteos": [],
        "iteraciones": []
    }

    for canonical, data in resumen.items():
        if data["count"] > 0:  # ⭐ solo incluir si apareció
            out["nombres"].append(canonical)
            out["conteos"].append(data["count"])
            out["iteraciones"].append(data["iters"])

    return out


In [ ]:
nombres_es = [
    # Masculinos
    "juan","jose","luis","carlos","jorge","pedro","manuel","miguel","angel","mario",
    "alejandro","ricardo","fernando","eduardo","roberto","antonio","francisco",
    "rafael","sergio","andres","adrian","victor","felipe","pablo","hector","diego",
    "alejo","daniel","alfredo","alberto","gustavo","ignacio","emilio","ramon",
    "german","julian","agustin","nicolas","rodrigo","samuel","tomas","oscar",
    "martin","roman","rubén","esteban","enrique","benjamin","isaac","barbara",

    # Femeninos
    "maria","ana","carmen","laura","patricia","adriana","diana","monica","luisa",
    "claudia","veronica","alejandra","sofia","isabel","gabriela","jessica","mariana",
    "valeria","fernanda","paula","beatriz","cecilia","silvia","rosa","elena","marta",
    "andrea","natalia","karla","angela","lucia","araceli","viviana","alicia",
    "estela","noemi","marisol","ivonne","marcela","rebeca","susana","ines",

    # Unisex o menos comunes
    "sam", "alex", "andrea", "cristian", "abril", "gael", "naomi", "emma", "lia",
]


# Pipeline

In [ ]:
prueba

,iteracion,PParticipante,id_sesion,texto
0,1,persona,596,¿Cuéntame cómo va a ser la audiencia de concil...
1,2,ai,596,💼 **Audiencia de Conciliación: Un Entorno de D...
2,3,unknown,596,1. **Presentación:** \n - El conciliador rev...
3,4,persona,596,Se considera despido si constantemente me amen...
4,5,ai,596,"Es comprensible que te sientas preocupado, Mar..."
5,6,persona,596,¿Qué sucede si no se llega a un acuerdo en la ...
6,7,ai,596,"María, es importante saber qué ocurre si la co..."
7,8,persona,596,¿Cuál es la diferencia entre un despido justif...
8,9,ai,596,Entender la diferencia entre un despido justif...
9,10,persona,596,Tengo contrato anual por cinco años que procede


In [ ]:
resultado = detectar_nombres_dicc(prueba, nombres_es)



In [ ]:
resultado

{'nombres': ['maria'], 'conteos': [3], 'iteraciones': [[5, 7, 11]]}

In [ ]:
data = []
for i in grabaciones:
  rest = {}
  pruebas = pd.read_json(f"gdrive/MyDrive/Conciliación laboral/Conversaciones ai/{i}.json")

  rest = detectar_nombres_dicc(pruebas,nombres_es)

  data.append(rest)

In [ ]:
id = []
for i in grabaciones:
  id.append(i)

In [ ]:
pd.DataFrame(data)

,nombres,conteos,iteraciones
0,[maria],[3],"[[5, 7, 11]]"
1,[],[],[]
2,[],[],[]
3,[rubén],[4],"[[2, 4, 6, 8]]"
4,[],[],[]
...,...,...,...
614,"[juan, eduardo]","[3, 3]","[[4, 6, 8], [4, 6, 8]]"
615,"[alejandra, jessica]","[3, 3]","[[4, 6, 8], [4, 6, 8]]"
616,[victor],[6],"[[2, 4, 6, 6, 8, 8]]"
617,"[maria, fernanda]","[6, 6]","[[2, 4, 9, 11, 13, 17], [2, 4, 9, 11, 13, 17]]"


In [ ]:
pd.DataFrame(data).to_json("gdrive/MyDrive/Conciliación laboral/data.json",
    orient="records",
      force_ascii=False,
      indent=2)

In [ ]:
df_datos = pd.DataFrame(data)

In [ ]:
df_datos["id"] = id

In [ ]:
df_datos

,nombres,conteos,iteraciones,id
0,[maria],[3],"[[5, 7, 11]]",480
1,[],[],[],479
2,[],[],[],478
3,[rubén],[4],"[[2, 4, 6, 8]]",477
4,[],[],[],476
...,...,...,...,...
614,"[juan, eduardo]","[3, 3]","[[4, 6, 8], [4, 6, 8]]",1027
615,"[alejandra, jessica]","[3, 3]","[[4, 6, 8], [4, 6, 8]]",1026
616,[victor],[6],"[[2, 4, 6, 6, 8, 8]]",1025
617,"[maria, fernanda]","[6, 6]","[[2, 4, 9, 11, 13, 17], [2, 4, 9, 11, 13, 17]]",1024


In [ ]:
test = pd.read_json("gdrive/MyDrive/Conciliación laboral/Conversaciones ai/1023.json")

In [ ]:
print(test.iloc[1].texto)

🎯 **Descifrando el Despido: Justificado vs Injustificado**

Cuando una persona trabajadora enfrenta un despido, es crucial entender si este se considera justificado o injustificado, ya que eso afecta lo que puede reclamar. Vamos a verlo de manera sencilla:

## 📋 Puntos Clave

1. **Despido Justificado**
   - Un despido es considerado justificado cuando ocurre por alguna de las razones que se mencionan en el artículo 47 de la Ley Federal del Trabajo. Algunos ejemplos son:
     - *Robar o dañar deliberadamente propiedad de la empresa.*
     - *No acatar órdenes sin un motivo válido.*
     - *Faltar al trabajo injustificadamente seguido.*
   - En estos casos, la empresa debe informar al trabajador sobre las razones del despido.

2. **Despido Injustificado**
   - Si el empleador no puede demostrar que se cumplieron estas razones o no avisa al trabajador de las causas, se presume que el despido es injustificado.
   - En esta situación, el trabajador puede reclamar ciertas prestaciones adicio

In [ ]:
df_datos.to_json("gdrive/MyDrive/Conciliación laboral/data.json",
    orient="records",
      force_ascii=False,
      indent=2)